<a href="https://colab.research.google.com/github/emmarogge/cs280r/blob/master/classification_project_key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2: Text classification with Colab and PyTorch

Emma Rogge, Tasha Schoenstein & Zilin Ma

## Set up

###Import relevant libraries and dependencies

In [1]:
import torch
import torch.nn as nn
from torch import optim
from torchtext import data
import math
import os
from collections import Counter

## GPU check, make sure to set runtime type to "GPU"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [2]:
## Read in data files
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.nl
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.sql
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/test.nl
!wget https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/test.sql

--2019-12-07 16:25:57--  https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.nl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 281581 (275K) [text/plain]
Saving to: ‘train.nl’

train.nl            100%[===================>] 274.98K  --.-KB/s    in 0.03s   

2019-12-07 16:25:57 (9.43 MB/s) - ‘train.nl’ saved [281581/281581]

--2019-12-07 16:25:58--  https://raw.githubusercontent.com/sriniiyer/nl2sql/master/data/atis/train.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2840349 (2.7M) [text/plain]
Saving to: ‘train.

##Data format

We're going to use `torchtext` to handle processing the data. This library is useful for processing and batching text data in Python. More information on `torchtext` can be found [in this tutorial](https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/).

###Implement torchtext Dataset
Implement the class below to prepare the data for classification.

#### Hints:
- Consider utilizing PyTorch's `preprocess` method on your text field objects--this method tokenizes the content of a text field object using `spacy`.
- Start by populating a list with each processed, tokenized query in your dataset.
- Each text field object in your list should then have its `label` field populated with the appropriate label.
- Leverage the `__init__` method of the parent class (`pytorch.utils.data.Dataset`).

---
### Bag-of-Words Text Representation
####Your Naive Bayes, logistic regression and MLP classifiers will use a bag of words representation for the data. The `torchtext` iterators output tokenized natural language, which you must convert into a bag of words representation. It is highly recommended that you make use of the `Example` class ([documentation found here](https://github.com/pytorch/text/blob/master/torchtext/data/example.py)). In this class, store the text, the label and the bag-of-words representation for each datum in your dataset.
---
####HINT: Your vocabulary should be derived ONLY from your training set, not your entire dataset. Make certain that your bag-of-words representations account for this. You may have unknown words in your test set and we leave it up to you to decide the best way of handling this.

In [0]:
## Convert to standard format
class ATIS(data.Dataset):
    dirname = 'data'
    name = 'atis'
    _vocab = {}
    _labels = []
    def __init__(self, path, text_field, label_field, bow_field, vocab, **kwargs):
        """Create an ATIS dataset instance given a path and fields.
        Arguments:
            path: Path to the data file
            text_field: The field that will be used for text data.
            label_field: The field that will be used for label data.
            bow_field: The field that will be used for bag-of-words data.
            vocab: dictionary mapping vocabulary words to unique indices **Optional**
            Remaining keyword arguments: Passed to the constructor of
                data.Dataset.
        """
        if (vocab is None): 
          fields = [('text', text_field), ('label', label_field), ('bow', bow_field)]
          examples = []
          # Get text
          with open(path+'.nl', 'r') as f:
            for line in f:
              ex = data.Example()
              # PyTorch's `preprocess` automatically does spacy tokenization
              ex.text = text_field.preprocess(line.strip()) 
              examples.append(ex)
            self._examples = examples
          
          # Map each vocab word to unique index 
          word_to_ix = {}
          for ex in self._examples:
            for word in ex.text:
              if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
          self._vocab = word_to_ix
          print("Vocab Size: {}".format(len(self._vocab)))

          # Get labels & bow representation
          with open(path + '.sql', 'r') as f:
              for i, line in enumerate(f):
                # Get label for text
                label = self._get_label_from_query(line.strip())
                self._examples[i].label = label
                self._labels.append(label)

                # Get bag-of-words for text
                text = self._examples[i].text
                vec = torch.zeros(len(word_to_ix))
                for w in text:
                  vec[self._vocab[w]] += 1
                bow = vec.view(1, -1)
                self._examples[i].bow = bow
          print("Loading dataset {} - {} examples".format(path, len(self._examples)))
          super(ATIS, self).__init__(self._examples, fields, **kwargs)
        
        else:
          words_outside_vocab =  0
          fields = [('text', text_field), ('label', label_field), ('bow', bow_field)]
          examples = []
          # Get text
          with open(path+'.nl', 'r') as f:
            for line in f:
              ex = data.Example()
              # PyTorch's `preprocess` automatically does spacy tokenization
              ex.text = text_field.preprocess(line.strip()) 
              examples.append(ex)
            self._examples = examples

          # Get labels & bow representation
          word_to_ix = vocab
          with open(path + '.sql', 'r') as f:
              for i, line in enumerate(f):
                # Get label for text
                label = self._get_label_from_query(line.strip())
                self._examples[i].label = label

                # Get bag-of-words for text
                text = self._examples[i].text
                vec = torch.zeros(len(word_to_ix))
                for w in text:
                  if w not in word_to_ix:
                    words_outside_vocab += 1
                    continue;
                  else:
                    vec[word_to_ix[w]] += 1
                bow = vec.view(1, -1)
                self._examples[i].bow = bow

          print("Loading dataset {} - {} examples".format(path, len(self._examples)))
          print("{} words outside of training vocabulary for {} dataset".format(words_outside_vocab, path))
          super(ATIS, self).__init__(self._examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex):
        return len(ex.text)

    # Simple function to get question labels from query
    def _get_label_from_query(self, query):
        parts = query.split(' ')
        if parts[1] == 'DISTINCT':
            label = parts[2]
        else:
            label = parts[1]
        
        if '.' in label:
            label = label.split('.')[-1]
        
        return label
    
    # Return labels
    def get_labels(self):
      return list(set(self._labels))
    
    # Return vocabulary
    def get_vocab(self):
      return self._vocab

    @classmethod
    def splits(cls, text_field, label_field, bow_field, path='./',
               train='train', validation='dev', test='test',
               **kwargs):
        """Create dataset objects for splits of the ATIS dataset.
        Arguments:
            text_field: The field that will be used for the sentence.
            label_field: The field that will be used for label data.
            bow_field: The field that will be used for bag-of-words data.
            root: The root directory that the dataset's zip archive will be
                expanded into; therefore the directory in whose trees
                subdirectory the data files will be stored.
            train: The filename of the train data. Default: 'train.txt'.
            validation: The filename of the validation data, or None to not
                load the validation set. Default: 'dev.txt'.
            test: The filename of the test data, or None to not load the test
                set. Default: 'test.txt'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.
        """
        # Get BOW representation for train split.
        train_data = None if train is None else cls(
            os.path.join(path, train), text_field, label_field, bow_field, None, **kwargs)
        vocab = train_data.get_vocab()
        labels = train_data.get_labels()

        # Use vocabulary to transform val, test splits into BOW.
        val_data = None if validation is None else cls(
            os.path.join(path, validation), text_field, label_field, bow_field, vocab, **kwargs)
        test_data = None if test is None else cls(
            os.path.join(path, test), text_field, label_field, bow_field, vocab, **kwargs)
        return tuple(d for d in (labels, vocab, train_data, val_data, test_data)
                     if d is not None)

###Implement tortchtext Iterators

Next, we create instances of the `data.Field` class for the input (text) and output (labels) fields.
The `data.Field` class, include in PyTorch, contains common text-processing datatypes that can be converted to tensors.


In [0]:
# We set `batch_first` = True to ensure the data is batched before it is processed.
TEXT = data.Field(lower=True, include_lengths=False, batch_first=True, tokenize="spacy") 
LABEL = data.Field(sequential=False, unk_token=None)
BOW = data.Field(batch_first=True)


We will use the `ATIS.splits` class method to build the `ATIS` instances for train and test data. This method splits the data into either two (train & test) or three (train, test, validation) subsets.

In [662]:
# Make splits for data
labels, vocab, train, test = ATIS.splits(TEXT, LABEL, BOW, validation=None)

Vocab Size: 860
Loading dataset ./train - 4379 examples
Loading dataset ./test - 448 examples
29 words outside of training vocabulary for ./test dataset


Once the data is processed we build the vocabulary and then construct iterators which loop over the datasets in batches. This will be important for SGD for logistic regression and for other models later in the course.

In [0]:
# Make iterator for splits
BATCH_SIZE = 32
train_iter, test_iter = data.BucketIterator.splits(
    (train, test), batch_size=BATCH_SIZE, device=device)

## Establish a majority baseline

By defining a lower bound on performance, we know at minimum what to expect from any reasonable system. A simple baseline for classification tasks is to measure the accuracy of prediction when the most common class is always predicted. 

**Write code in the cell below that, given train and test data, prints information concerning the majority baseline.**

In [626]:
def majority_baseline_accuracy(train, test):
  # Find majority on training data
  counts = Counter()
  for ex in train:
      counts[ex.label] += 1

  most_common = counts.most_common(1)[0][0]
  print("Most common label: {}".format(most_common))
  # Evaluate accuracy on test data
  test_counts = Counter()
  for ex in test:
      test_counts[ex.label] += 1

  total_count = len(list(test_counts.elements()))
  most_common_count = test_counts[most_common]
  print('Count of most common label:', most_common_count)
  print('Count of total things labelled:', total_count)
  print('Portion of labels that are the most common one:', most_common_count/total_count)

majority_baseline_accuracy(train, test)

Most common label: flight_id
Count of most common label: 306
Count of total things labelled: 448
Portion of labels that are the most common one: 0.6830357142857143


# Naive Bayes

Naive Bayes classification is based on the "naive" assumption that all features are independent. This dramatically reduces the number of parameters required for Bayesian classification, which utilizes Bayes' Theorem which utilizes known information ($P(Y)$, $P(X)$ and $(P(X_i|Y)$) to obtain the desired unknown probability of $P(Y|X)$. This is evaluated for each possible label and the label with greatest likelihood is the prediction for a given text. 

---
Let $ c_{NB} $ be the maximum value in a vector containing the conditional probabilities of label $c$ given each word in the text. Then, we can compute $c_{NB}$ by evaluating the probability of the label overall and the probability of the label given the presence of each word contained in a given text, as 
$$ c_{NB} = \text{argmax}_{c \in C} \left( \log P(c) + \sum_{w \in W}\log P(w|c) \right) $$

Where $c_{NB}$ is the naive Bayes classification of a bag of words, $C$ is the set of classifications, and $W$ is the bag of words.

We can calculate $P(c) = \frac{N_c}{N}$ where $N$ is the total number of data points in our training data and $N_c$ is the total number of data points in our training data with classification $c$. 

We can calculate $P(w_0 | c)$ using Laplace smoothing such that $$P(w_0 | c) = \frac{count(w_0, c) + 1}{\left( \sum_{w \in V} count(w,c)\right) + |V|}$$ where $V$ is the vocabulary.

----
##Below, implement the NaiveBayes class methods.
 

###1.  `train`: Populates the log probabilities table to contain $log(P(c))$ and $log(𝑃(𝑤_i|𝑐)) $ for each label for each word in the vocabulary.**
###2.   `evaluate_performance`: Evaluates the performance of the model on given datset and prints accuracy.

In [0]:
class NaiveBayes():
    def __init__ (self, texts, bows, labels, vocab):
      self.texts = texts
      self.bows = bows
      self.labels = labels
      self.vocab = vocab
      self.log_probs = {}
    
    def train(self, train):
      """
      Populates log probabilities table for training data.
      """
      for label in self.labels:
        self.log_probs[label] = {}

        # Calculate the log prior (logP(c))
        N = len(self.labels)
        Nc = sum(example.label == label for example in train.examples)
        self.log_probs[label]['log_prior'] = math.log(Nc / N)

        # Calculate the log likelyhood (logP(w | c)) for all words in vocab for each label
        self.log_probs[label]['log_likelihood'] = {}
        for word in vocab:
          count_wc = 0
          sum_count_wc = 0
          for example in train.examples:
            if example.label == label:
              count_wc += sum(token == word for token in example.text)
              sum_count_wc += len(example.text)
          Pwc = (count_wc + 1) / (sum_count_wc + len(vocab))
          self.log_probs[label]['log_likelihood'][word] = math.log(Pwc)
    
    def evaluate_performance(self, dataset):
      """
      Takes a dataset and prints the model's performance that dataset.
      """
      # Count the number of correct guesses
      correct_guesses = 0
      for example in dataset.examples:
       # For each example, find the score of each label 
        scores = {}
        for label in self.log_probs:
          class_prediction = self.log_probs[label]['log_prior']
          for word in example.text:
            if word in vocab:
              class_prediction += self.log_probs[label]['log_likelihood'][word]
          scores[label] = class_prediction

        # Find the maximum score to determine our guess for the label
        argmax = max(scores, key=scores.get)

        # If it matches the actual label, we guessed correctly!
        if argmax == dataset.examples[dataset.examples.index(example)].label:
          correct_guesses += 1

      # Print our accuracy
      print('Accuracy: ', correct_guesses / len(dataset.examples))
      return correct_guesses/len(dataset.examples)

## Putting it all together

If you have implemented the class methods, the following should result in a trained model.

In [667]:
# Bag-of-words vectors for each input query
# batch = next(iter(train_iter))

# Instantiate and train classifier
nb_classifier = NaiveBayes(TEXT, BOW, labels, vocab)
nb_classifier.train(train)

# Evaluate model performance
print("Train: ")
classifier.evaluate_performance(train)
print("Test: ")
classifier.evaluate_performance(test)

Train: 
Accuracy:  0.8942680977392099
Test: 
Accuracy:  0.84375


0.84375

# Logistic Regression

Unlike Naive Bayes, logistic regression calculates the conditional probabilities directly. If we let $c\in C$ be a label, $\mathbf{w} \in W$ be a bag-of-words representation of a natural language query, $\mathbf{d}$ be weights in the model tied to the compatability of $c$ and $\mathbf{w}$, and $f$ is $\mathbf{d}^T \mathbf{w}$, we use the softmax to get: 
$$ p(c|\mathbf{w}, \mathbf{d})= \frac{\exp (f(\mathbf{w},c,\mathbf{d}))}{\sum_{c'\in C}\exp(f(\mathbf{w},c',\mathbf{d}))}. $$

The weights are learned in the process of training by using a loss function--here the cross entropy loss--to compare the results produced by the current version of the model and the target results. 
---
##Below, implement the LogisticRegression class methods.
 

###1.  `train`: Trains the model for n epochs with provided optimizer and learning rate.
###2.   `evaluate_performance`: Evaluates the performance of the model on given datset and prints accuracy.



In [0]:
'''
Implement the initialization and forward step of the Logistic Regression model.
'''
class LogisticRegression(nn.Module):
    def __init__ (self, count_labels, bag_size):
        super (LogisticRegression, self).__init__ ()
        # Linear layer
        self.fc = nn.Linear(bag_size, count_labels)
        # Bias
        self.bias = torch.zeros(count_labels, requires_grad=True).to(device)
        
    def forward (self, input):
        # Apply the linear layer
        output = self.fc(input)
        # Add the bias and output the result
        output = (output + self.bias)
        return output

    '''
    Takes criterion, optimizer and # of epochs, and trains the model on the given data.
    '''
    def train (self, bow, criterion, optim, n_epochs = 8):
      for epoch in range (n_epochs):
        c_num = 0
        total = 0
        for index, batch in enumerate(bow):
          print("index: {} batch: {}\n".format(index, batch))
          # Initialize the optimizer
          optim.zero_grad()
          
          # Input and target
          input = input2bow(batch.text, len(TEXT.vocab), TEXT.vocab.stoi['<pad>'])
          target = batch.label.long()
          
          # Feed the input and hidden state to the model
          scores = self(input)

          # Compute the loss
          loss = criterion(scores, target)
          
          # Perform backpropogation
          loss.backward()
          optim.step()
          
          # Prepare to compute the accuracy
          predictions = torch.argmax(scores, dim=1)
          total += len(target)
          c_num += (predictions == target).sum().item()

          # Report the loss every 200 steps
          if index % 200 == 0:
              print ('Epoch :', epoch,
                    'Step: ', index,
                    'Loss: ', loss.item(),
                    'Accuracy:', float (c_num)/total)

    '''
    Takes a model & dataset, and returns accuracy of model on dataset.
    '''
    def evaluate_performance(self, dataset):
        # c_num = 0
        # total = 0

        # with torch.no_grad():
        #   data_iter = iter(dataset)
        #   for index, batch in enumerate(data_iter):
        #       # Input and target
        #       input = input2bow(batch.text, len(TEXT.vocab), TEXT.vocab.stoi['<pad>'])
        #       target = batch.label.long()

        #       # Feed the input and hidden state to the model 
        #       # then determine the index of the maximum value for each test item
        #       scores = self(input)
        #       predictions = torch.argmax(scores, dim=1)
  
        #       # Prepare to compute the accuracy
        #       total += len(target)
        #       print(predictions == target)
        #       c_num += (predictions == target).sum().item()

        # # Return the accuracy
        # return float (c_num)/total
          c_num = 0
          total = 0
          
          # Turn on eval mode
          # model.eval ()

          with torch.no_grad():
              for index, batch in enumerate(test_iter):
                  # Input and target
                  input = input2bow(batch.text, len(TEXT.vocab), TEXT.vocab.stoi['<pad>'])
                  target = batch.label.long()

                  # Feed the input and hidden state to the model then determine the 
                        # index of the maximum value for each test item
                  scores = model(input)
                  predictions = torch.argmax(scores, dim=1)
      
                  # Prepare to compute the accuracy
                  total += len(target)
                  c_num += (predictions == target).sum().item()

          # Return the accuracy
          return float (c_num)/total

## Putting it all together

If you have implemented the LogisticRegression class methods, the following should result in a trained model.

In [351]:
# Bag-of-words vectors for each input query
batch_bow = input2bow(batch.text, len(TEXT.vocab) - 1, TEXT.vocab.stoi['<pad>'])
print(len(batch.text))

# Instantiate classifier
#Subtract 1 because the bag of words representation removes the padding
linear_regression_model = LogisticRegression(LABEL, TEXT).to(device) 
print(linear_regression_model)
loss = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train classifier model on training split
linear_regression_model.train(train_iter, loss, optimizer)

# Evaluate model performance on training, test splits
print("Train:")
linear_regression_model.evaluate_performance(train_iter)
print("Test:")
linear_regression_model.evaluate_performance(test_iter)

TypeError: ignored

# Multilayer Perceptron

An MLP is composed of at least three fully connected layers of nodes, referred to as the input, output and "hidden" layers. Learning occurs by adjusting the connection weights between nodes based on the amount of error in the output compared to the prediction. 
---
Let the degree of error in an output node $j$ in the $n$th training query be $e_j(n) = d_j(n) - y_j(n)$, where $d$ is the true label and $y$ is the predicted label.

Then we can adjust the weights to minimize the entire output layer's cumulative error:
$$\mathcal{E}(n)=\frac{1}{2}\sum_j e_j^2(n)$$

The change in each weight according to gradient descent is 
$$\Delta w_{ji} (n) = -\eta\frac{\partial\mathcal{E}(n)}{\partial v_j(n)} y_i(n)$$.

---
##Implement the methods of the class MultiLayerPerceptron below.

In [0]:
import torch

class MultiLayerPerceptron(nn.Module):
  def __init__(self, label, text, n_hidden=128):
    super(MultiLayerPerceptron, self).__init__()
    self.label = label
    self.text = text
    self.input2hidden = nn.Linear(len(self.text.vocab), n_hidden)
    print(self.label.vocab.itos)
    print("len labels: {}".format(self.label.vocab.itos))
    self.hidden2output = nn.Linear(n_hidden, len(self.label.vocab.itos))
    self.softmax = nn.LogSoftmax()
    self.learning_rate = 0.01
    self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
    print("length of text.vocab = {}".format(len(self.text.vocab)))

  def forward(self, data):
    hidden = self.input2hidden(data)
    output = self.hidden2output(hidden)
    output = self.softmax(output)
    return output

  def train(self, bow, n_epochs):
      for epoch in range (n_epochs):
        curr_loss = 0.0
        c_num = 0
        total = 0
        for index, batch in enumerate(bow):
          
          # Zero the parameter gradients
          self.optimizer.zero_grad()
          
          # Input and target
          input = input2bow(batch.text, len(self.text.vocab), self.text.vocab.stoi['<pad>'])
          target = batch.label.long()
          print(batch.label.long())
          print("len text vocab {}".format(len(self.text.vocab)))
          print("len label vocab {}".format(len(self.label.vocab)))
          
          # Forward step
          predictions = self(input)
          criterion = nn.NLLLoss()

          # Compute loss
          loss = criterion(predictions, target)
          total += len(target)
          c_num += (predictions == target).sum().item()
          
          # Backward step
          loss.backward()
          optimizer.step()

          # Report the loss every 200 steps
          curr_loss += loss.item()
          if index % 200 == 0:
              print ('Epoch :', epoch,
                    'Step: ', index,
                    'Loss: ', loss.item(),
                    'Accuracy:', float (c_num)/total)

  def evaluate_performance(self, data):
    c_num = 0
    total = 0
    with torch.no_grad():
      for index, batch in enumerate(data):
        # Input and target
        input = input2bow(batch.text, len(self.text.vocab) - 1, self.text.vocab.stoi['<pad>'])
        target = batch.label.long()

        # Feed the input and hidden state to the model then determine the 
        # index of the maximum value for each test item
        scores = self(input)
        predictions = torch.argmax(scores, dim=1)

        # Prepare to compute the accuracy
        total += len(target)
        c_num += (predictions == target).sum().item()

        # Return the accuracy
        print("Accuracy: {}".format(float(c_num)/total))
        return float (c_num)/total

In [429]:
# Bag-of-words vectors for each input query
batch_bow = input2bow(batch.text, len(TEXT.vocab), TEXT.vocab.stoi['<pad>'])

# Instantiate classifier
mlp_classifier = MultiLayerPerceptron(LABEL, TEXT).to(device)
print(mlp_classifier)

# Train classifier model on training split
mlp_classifier.train(batch_bow, 5)

# Evaluate model performance on training, test splits
print("Train:")
mlp_classifier.evaluate_performance(train_iter)
print("Test:")
mlp_classifier.evaluate_performance(test_iter)

Vocab Size: 862 Batch X: 32 Y: 21
Rows in Batch: 32
batch_bow shape: torch.Size([32, 861])
['flight_id', 'fare_id', 'transport_type', 'airline_code', 'aircraft_code', 'departure_time', 'fare_basis_code', 'airport_code', 'count', 'state_code', 'booking_class', 'ground_fare', 'restriction_code', 'arrival_time', 'miles_distant', 'city_code', 'meal_code', 'advance_purchase', 'basic_type', 'flight_number', 'meal_description', 'minutes_distant', 'airport_location', 'time_elapsed', 'day_name', 'stop_airport', 'stops', 'city_name', 'minimum_connect_time', 'time_zone_code']
len labels: ['flight_id', 'fare_id', 'transport_type', 'airline_code', 'aircraft_code', 'departure_time', 'fare_basis_code', 'airport_code', 'count', 'state_code', 'booking_class', 'ground_fare', 'restriction_code', 'arrival_time', 'miles_distant', 'city_code', 'meal_code', 'advance_purchase', 'basic_type', 'flight_number', 'meal_description', 'minutes_distant', 'airport_location', 'time_elapsed', 'day_name', 'stop_airport',

AttributeError: ignored